In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models import Word2Vec, KeyedVectors
import tensorflow as tf
from keras.models import Sequential, load_model
from keras import layers
from keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional, Lambda, Flatten
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from sklearn.manifold import TSNE
import keras
from keras import backend as K

In [2]:
data = pd.read_csv('C:\\Users\\user1\\Documents\\repo\\dialog act RNN\\MRDA\\full_set.csv')
print(data.shape)
data.head(10)

(108201, 6)


,Unnamed: 0,Speaker,Utterances,Basic,General,Full
0,0,fe016,so um,F,fh,fh
1,1,fe016,i was going to try to get out of here like in ...,S,s,rt
2,2,fe016,um,F,fh,fh
3,3,fe016,because i really appreciate people coming.,S,s,s
4,4,fe016,and the main thing that i was going to ask peo...,S,s,s
5,5,fe016,so anything that transcribers or discourse cod...,S,s,e
6,6,fe016,so we have this um,D,fh,fh
7,7,fe016,i think a starting point is clearly the the ch...,S,s,s
8,8,fe016,which don brought a copy of.,S,s,e
9,9,me011,yeah.,B,b,b


In [3]:
data=data.drop(columns=["Unnamed: 0"])
data.head()

,Speaker,Utterances,Basic,General,Full
0,fe016,so um,F,fh,fh
1,fe016,i was going to try to get out of here like in ...,S,s,rt
2,fe016,um,F,fh,fh
3,fe016,because i really appreciate people coming.,S,s,s
4,fe016,and the main thing that i was going to ask peo...,S,s,s


In [4]:
data['Utterances'] = data['Utterances'].apply(lambda x: x.lower())
data['Utterances'] = data['Utterances'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
data['Utterances'].head()

0                                                so um
1    i was going to try to get out of here like in ...
2                                                   um
3            because i really appreciate people coming
4    and the main thing that i was going to ask peo...
Name: Utterances, dtype: object

In [5]:
print((data.groupby('Full')['Full'].agg('count')).size)
data.groupby('Full')['Full'].agg('count')

52


Full
%       3103
2        841
aa      5898
aap      219
am       349
ar       908
arp      150
b      15013
ba      2216
bc        51
bd       387
bh       154
bk      7177
br       236
bs       141
bsc      150
bu      2091
by        11
cc       371
co       674
cs      2662
d       1805
df      3724
e       3200
f        128
fa       259
fe       307
fg      3091
fh      8362
ft       119
fw         6
g         87
h        792
j        463
m        293
na      1112
nd       483
ng       351
no       828
qh       214
qo        74
qr       127
qrr      345
qw       951
qy       669
r        208
rt      3101
s      33472
t        253
t1       198
t3       165
tc       212
Name: Full, dtype: int64

In [6]:
y = pd.get_dummies(data['Full']).values
[print(data['Full'][i],y[i]) for i in range (0,5)]

fh [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
rt [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
fh [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
s [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
s [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


[None, None, None, None, None]

In [7]:
X = data['Utterances']
tokenizer = Tokenizer(num_words = 4000)
tokenizer.fit_on_texts(X)
sequence = tokenizer.texts_to_sequences(X)

In [8]:
index_of_words = tokenizer.word_index
print(len(index_of_words))

11113


In [9]:
X = pad_sequences(sequence , maxlen = 77 , padding='post')
X[0]

array([11, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0])

In [10]:
f = open("C:\\Users\\user1\\Downloads\\glove.6B\\glove.6B.300d.txt",  encoding="utf8")
embedd_index = {}
for line in f:
    val = line.split()
    word = val[0]
    coff = np.asarray(val[1:],dtype = 'float')
    embedd_index[word] = coff

f.close()
print('Found %s word vectors.' % len(embedd_index))

Found 400000 word vectors.


In [11]:
embedd_index['good']

array([-1.3602e-01, -1.1594e-01, -1.7078e-02, -2.9256e-01,  1.6149e-02,
        8.6472e-02,  1.5759e-03,  3.4395e-01,  2.1661e-01, -2.1366e+00,
        3.5278e-01, -2.3909e-01, -2.2174e-01,  3.6413e-01, -4.5021e-01,
        1.2104e-01, -1.5596e-01, -3.8906e-02, -2.9419e-03,  1.6009e-02,
       -1.1620e-01,  3.8680e-01,  3.5109e-01,  9.7426e-02, -1.2425e-02,
       -1.7864e-01, -2.3259e-01, -2.6960e-01,  4.1083e-02, -7.6194e-02,
       -2.3362e-01,  2.0919e-01, -2.7264e-01,  5.4967e-02, -1.8055e+00,
        5.6348e-01, -1.2778e-01,  2.3147e-01, -5.8820e-03, -2.6630e-01,
        4.1187e-01, -3.7162e-01, -2.0600e-01, -1.9619e-01, -4.3945e-03,
        1.2513e-01,  4.6638e-01,  4.5159e-01, -1.5000e-01,  5.9589e-03,
        5.9070e-02, -4.1440e-01,  6.1035e-02, -2.1117e-01, -4.0988e-01,
        5.6393e-01,  2.3021e-01,  2.7240e-01,  4.9364e-02,  1.4239e-01,
        4.1841e-01, -1.3983e-01,  3.4826e-01, -1.0745e-01, -2.5002e-01,
       -3.2554e-01,  3.3343e-01, -3.5617e-01,  2.0442e-01,  1.44

In [12]:
embedding_matrix = np.zeros((len(index_of_words) + 1, 300))

tokens = []
labels = []

for word,i in index_of_words.items():
    temp = embedd_index.get(word)
    if temp is not None:
        embedding_matrix[i] = temp
        
embedding_matrix.shape

(11114, 300)

In [13]:
embedd_layer = Embedding(len(index_of_words) + 1 , 300 , input_length = 77 , weights = [embedding_matrix])

In [14]:
MAX_LENGTH_PER_SENTENCE=77
units = 128
encoder_input = keras.Input(shape=(MAX_LENGTH_PER_SENTENCE))
x = layers.Embedding(len(index_of_words) + 1 , 300 , input_length = 77 , weights = [embedding_matrix])(encoder_input)
                              
activations = layers.Bidirectional(layers.LSTM(units, dropout=0.3, recurrent_dropout=0.2))(x)

attention = layers.Dense(1, activation='tanh')(activations)
attention = layers.Flatten()(attention)
attention = layers.Activation('softmax')(attention)
attention = layers.RepeatVector(units*2)(attention)
attention = layers.Permute((2, 1))(attention)

sent_representation = layers.Multiply()([activations, attention])
sent_representation = layers.Lambda(lambda X: K.sum(X, axis=-2), output_shape=(units*2,))(sent_representation)


probabilities = layers.Dense(52, activation='softmax')(sent_representation)


encoder = keras.Model(inputs=[encoder_input], outputs=[probabilities],name='encoder')

encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 77)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 77, 300)      3334200     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 256)          439296      embedding_1[0][0]                
__________________________________________________________________________________________________
dense (Dense)                   (None, 1)            257         bidirectional[0][0]              
____________________________________________________________________________________________

In [15]:
encoder.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [17]:
encoder.fit(X_train, y_train, epochs=3, batch_size=32, verbose=1)

Epoch 1/3
2705/2705 [==============================] - 1045s 385ms/step - loss: 1.5993 - accuracy: 0.5662
Epoch 2/3
2705/2705 [==============================] - 1024s 378ms/step - loss: 1.4085 - accuracy: 0.5933
Epoch 3/3
2705/2705 [==============================] - 955s 353ms/step - loss: 1.3400 - accuracy: 0.6035


In [21]:
score = encoder.evaluate(X_test, y_test)

677/677 [==============================] - 26s 39ms/step - loss: 1.4116 - accuracy: 0.5859


In [22]:
print(X_test[1])
print(y_test[1])

[ 15   1 517   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]


In [23]:
a = ["My name is rahul"]
a = tokenizer.texts_to_sequences(a)
a = np.array(a)
a = pad_sequences(a, padding='post', maxlen=77)
prediction = encoder.predict(np.array(a))
print("Shape of example ", a.shape)
print("Shape of predicted ", prediction.shape)
print("Prediction", prediction)
print("Max value", np.max(prediction))
print("Index of the max value " , prediction.argmax(1))

Shape of example  (1, 77)
Shape of predicted  (1, 52)
Prediction [[6.49838969e-02 2.89740320e-03 6.86646765e-03 3.20238504e-03
  3.18196253e-03 4.41392505e-04 4.43256984e-04 2.41305475e-04
  1.54551817e-03 5.53842809e-04 2.98100105e-03 5.05947741e-04
  4.35007922e-03 7.80498842e-03 1.03675376e-03 1.82005402e-03
  5.36284503e-03 6.58679754e-04 2.84166588e-03 3.74936988e-03
  6.68847933e-03 4.21740580e-03 1.06671415e-02 1.53272916e-02
  1.68353887e-04 2.88603973e-04 1.31032197e-03 2.66406173e-03
  1.32943632e-03 1.06409374e-04 2.86152881e-05 1.77746944e-04
  6.39094680e-04 2.36546732e-02 2.97787553e-03 7.31115937e-02
  8.22441652e-03 2.22205860e-03 2.50555365e-03 2.05044798e-03
  3.64894659e-04 1.74723769e-04 1.32400062e-04 2.68715434e-03
  4.51110443e-03 8.39057285e-03 2.04737354e-02 6.49385333e-01
  1.20557677e-02 9.44228936e-03 1.14397816e-02 7.11387303e-03]]
Max value 0.64938533
Index of the max value  [47]
